In [ ]:
# load creds
%load_ext dotenv
%dotenv

In [ ]:
from xcube_sh.cube import open_cube
from xcube_sh.config import CubeConfig
from xcube.core.maskset import MaskSet

from osgeo import gdal, #gdal_array, ogr

import IPython.display

%matplotlib inline

General Parameters

In [ ]:
dataset = "S2L2A"
spatial_res = 0.00009 # 10m
band_names = ["B02", "B03", "B04", "B08", "B11", "SCL"]
time_period = "1D"